# Geometric Feature Computation

In [3]:
import pandas as pd
import numpy as np

## Loading The Dataset With Features

In [177]:
scale = "1point0"

In [178]:
file_path = f"CloudCompare/area2_cov_multi-nbd-{scale}-features.csv"

data = pd.read_csv(file_path)
data.head()

,//X,Y,Z,Scalar field,Eigenvalues sum (1),Omnivariance (1),Eigenentropy (1),Anisotropy (1),Planarity (1),Linearity (1),Sphericity (1),Verticality (1),Nx,Ny,Nz
0,0.001387,1.10728,0.017115,2.0,0.10524,0.01319,0.31369,0.98523,0.79733,0.18790,0.01477,0.00002,0.001091,0.125512,0.992091
1,0.001387,1.10974,0.017115,2.0,0.10538,0.01320,0.31396,0.98526,0.79686,0.18839,0.01474,0.00002,0.001052,0.081042,0.996710
2,0.001541,1.11221,0.017115,2.0,0.10552,0.01321,0.31424,0.98528,0.79624,0.18904,0.01472,0.00002,0.001052,0.081042,0.996710
3,0.001541,1.11221,0.017115,2.0,0.10552,0.01321,0.31424,0.98528,0.79624,0.18904,0.01472,0.00002,0.001052,0.081042,0.996710
4,0.003545,1.11714,0.017115,2.0,0.10593,0.01324,0.31502,0.98535,0.79561,0.18974,0.01465,0.00002,0.001381,0.697373,0.716707


In [179]:
features = ['x', 'y', 'z', 'class', 'eigenvalues_sum', 'omnivariance', 'eigentropy', 'anisotropy', 'linearity', 'sphericity', 'planarity', 'verticality']
geometric_features = ['eigenvalues_sum', 'omnivariance', 'eigentropy', 'anisotropy', 'linearity', 'sphericity', 'planarity', 'verticality']

Rename columns for convenience

In [180]:
scale = f"({scale.split('point')[0] + '.' + scale.split('point')[1]})"
if scale == "(1.0)":
    scale = "(1)"

In [181]:
data = data.rename(columns={'//X': 'x', 'Y': 'y', 'Z': 'z', 'Scalar field': 'class', f"Eigenvalues sum {scale}": 'eigenvalues_sum', f"Omnivariance {scale}": 'omnivariance', f"Eigenentropy {scale}": 'eigentropy', f"Anisotropy {scale}": 'anisotropy', f"Linearity {scale}": 'linearity', f"Sphericity {scale}": 'sphericity', f"Planarity {scale}": 'planarity', f"Verticality {scale}": 'verticality'})

In [182]:
# Function to print the percentage of null values in each column
def print_null_percentage(df):
    total_rows = len(df)
    for column in df.columns:
        null_count = df[column].isnull().sum()
        percentage_null = (null_count / total_rows) * 100
        print(f"Column '{column}': {percentage_null:.2f}% null values")

In [183]:
print_null_percentage(data)

Column 'x': 0.00% null values
Column 'y': 0.00% null values
Column 'z': 0.00% null values
Column 'class': 0.00% null values
Column 'eigenvalues_sum': 0.00% null values
Column 'omnivariance': 0.00% null values
Column 'eigentropy': 0.00% null values
Column 'anisotropy': 0.00% null values
Column 'planarity': 0.00% null values
Column 'linearity': 0.00% null values
Column 'sphericity': 0.00% null values
Column 'verticality': 0.00% null values
Column 'Nx': 0.00% null values
Column 'Ny': 0.00% null values
Column 'Nz': 0.00% null values


In [184]:
data.drop(['Nx', 'Ny', 'Nz'], axis=1, inplace=True)

## Computing Class-Wise Mean and Variance

In [185]:
grouped = data.groupby(data['class'])
averages = grouped.mean()
variances = grouped.var()
averages
# variances

,x,y,z,eigenvalues_sum,omnivariance,eigentropy,anisotropy,planarity,linearity,sphericity,verticality
class,,,,,,,,,,,
1.0,0.561818,1.369313,0.017125,0.119789,0.014106,0.339354,0.988357,0.669069,0.319281,0.011643,0.000034
2.0,0.496890,1.374248,0.017118,0.118266,0.014000,0.336603,0.988134,0.672568,0.315561,0.011866,0.000040
5.0,0.539540,1.364338,0.064347,0.119638,0.014094,0.339064,0.988342,0.668385,0.319950,0.011658,0.000034
8.0,0.512437,1.327641,0.041748,0.119446,0.014082,0.338741,0.988311,0.669778,0.318527,0.011689,0.000033


In [186]:
# Function to compute the covariance matrix
def compute_covariance_matrix(data, regularization=0):
    cov_matrix = np.cov(data, rowvar=False)
    cov_matrix += regularization * np.eye(cov_matrix.shape[0])
    return cov_matrix

# Function to fit the model
def fit(x_train, y_train):
    y_train = y_train.ravel()
    m = y_train.shape[0] 
    x_train = x_train.reshape(m, -1)
    input_feature = x_train.shape[1] # Number of input feature. In our case it's 4
    class_label = 9
    mu = np.zeros((class_label, input_feature))
    sigma = np.zeros((class_label, input_feature, input_feature))
    phi = np.zeros(class_label)

    for label in range(class_label):
        # Seperate all the training data for a single class
        indices = (y_train == label)
        
        phi[label] = float(np.sum(indices)) / m
        mu[label] = np.mean(x_train[indices, :], axis=0)
        # Instead of writting the equation we used numpy covariance function. 
        sigma[label] = compute_covariance_matrix(x_train[indices, :])
    
    return phi, mu, sigma

In [187]:
# Dropping any null values
# data = data.dropna()

## Fitting The Gaussian Discriminant Analysis (GDA) Model

In [188]:
x = data[geometric_features]
y = data[['class']]

x = x.values
y = y.values

In [189]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [190]:
data.shape

(266675, 12)

In [191]:
phi, mu, sigma = fit(x_train, y_train)
phi

c:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\Vidhish17\AppData\Local\Temp\ipykernel_4660\930009657.py:3: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_matrix = np.cov(data, rowvar=False)
c:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Vidhish17\Desktop\RE\gvcl\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


array([0.        , 0.14535952, 0.19212993, 0.        , 0.        ,
       0.25900441, 0.        , 0.        , 0.40350614])

In [192]:
for label in [int(i) for i in data['class'].unique()]:
    print(np.linalg.eigvals(sigma[label]))
    print(np.linalg.det(sigma[label]))

[3.57725302e-03 4.46485014e-05 5.49697767e-09 3.74644902e-09
 6.54979821e-10 1.08925849e-11 7.16087011e-12 2.68263755e-14]
4.50803229488527e-69
[1.59978556e-03 2.63266674e-05 2.99291496e-09 2.14256002e-09
 2.65445188e-10 8.67472790e-12 4.68453711e-12 2.43879705e-14]
7.104902175749669e-71
[1.40057513e-03 1.97680463e-05 3.00999151e-09 1.51169445e-09
 2.35017491e-10 8.90622886e-12 3.83441100e-12 2.57598144e-14]
2.604563067639865e-71
[1.61647108e-03 2.52874586e-05 2.70077971e-09 1.80270244e-09
 3.80596594e-10 8.56490638e-12 4.30931266e-12 1.71915105e-14]
4.806134130669556e-71


In [193]:
def multivariate_gaussian_pdf(x, mean, cov):
    d = mean.shape[0]
    exponent = -0.5 * np.dot(np.dot((x - mean).T, np.linalg.inv(cov)), (x - mean))
    prefactor = 1 / np.sqrt((2 * np.pi) ** d * np.linalg.det(cov))
    #print("{} * {} = {}".format(prefactor, np.exp(exponent), prefactor* np.exp(exponent)))
    return np.exp(exponent)

### Checking If The Obtained Matrices are Positive Semidefinite

In [194]:
def is_positive_semidefinite(matrix):
    eigenvalues, _ = np.linalg.eig(matrix)
    print(eigenvalues)
    return np.all(eigenvalues >= 0)

for i in [int(i) for i in data['class'].unique()]:
    matrix = sigma[i]  # Example matrix
    positive_semidefinite = is_positive_semidefinite(matrix)
    if positive_semidefinite:
        print(f"Class {i}: The matrix is positive semidefinite.")
    else:
        print(f"Class {i}: The matrix is NOT positive semidefinite.")
    print("\n")

[3.57725302e-03 4.46485014e-05 5.49697767e-09 3.74644902e-09
 6.54979821e-10 1.08925849e-11 7.16087011e-12 2.68263755e-14]
Class 2: The matrix is positive semidefinite.


[1.59978556e-03 2.63266674e-05 2.99291496e-09 2.14256002e-09
 2.65445188e-10 8.67472790e-12 4.68453711e-12 2.43879705e-14]
Class 8: The matrix is positive semidefinite.


[1.40057513e-03 1.97680463e-05 3.00999151e-09 1.51169445e-09
 2.35017491e-10 8.90622886e-12 3.83441100e-12 2.57598144e-14]
Class 1: The matrix is positive semidefinite.


[1.61647108e-03 2.52874586e-05 2.70077971e-09 1.80270244e-09
 3.80596594e-10 8.56490638e-12 4.30931266e-12 1.71915105e-14]
Class 5: The matrix is positive semidefinite.




In [195]:
x_test.shape

(53335, 8)

## Computing Feature Densities

In [196]:
# feature_densities = []
# for i in range (x_test.shape[0]):
#     feature_density = 0
#     for label in [int(i) for i in data['class'].unique()]:
#         feature_density += phi[label]*multivariate_gaussian_pdf(x_test[i], mu[label], sigma[label])
#     feature_densities.append([x_test[i], feature_density])

feature_densities = []
labels = [int(i) for i in data['class'].unique()]

for i in range (x_test.shape[0]):
    rel_probs = []
    deno = 0
    for label in labels:
        x = multivariate_gaussian_pdf(x_test[i], mu[label], sigma[label])
        deno += x
        rel_probs.append(x)
    probs = [x/deno for x in rel_probs]
    feature_density = 0
    for j in range (len(labels)):
        feature_density += phi[labels[j]]*probs[j]
    feature_densities.append([x_test[i], feature_density])

C:\Users\Vidhish17\AppData\Local\Temp\ipykernel_4660\2168344499.py:18: RuntimeWarning: invalid value encountered in scalar divide
  probs = [x/deno for x in rel_probs]


In [197]:
print(len(feature_densities))
feature_densities[0]

53335


[array([1.1311e-01, 1.3480e-02, 3.2556e-01, 9.8820e-01, 4.0640e-01,
        1.1800e-02, 5.8181e-01, 1.9000e-04]),
 0.19212995088078952]

In [198]:
from tensorflow import keras
import tensorflow as tf

In [199]:
X = data[geometric_features].values
y = data['class'].values  # Get class labels

# Filter out classes not present in the dataset
num_classes = len(data['class'].unique())  # Update to the number of present classes (1, 2, 5, 8)
classes_present = [int(i) for i in data['class'].unique()]  # Classes present in the dataset
class_mapping = {cls: i for i, cls in enumerate(classes_present)}
y_mapped = np.array([class_mapping[cls] for cls in y])

# Convert class labels to one-hot encoding
y_onehot = tf.one_hot(y_mapped, depth=num_classes)


# Define the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(len(geometric_features),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(4, activation='softmax')  # Output layer with softmax activation for the present classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y_onehot, epochs=10, batch_size=32, validation_split=0.2)

softmax_probs = model.predict(x_test)
entropy = -np.sum(softmax_probs * np.log(softmax_probs), axis=-1)

print("Per-point softmax entropy:", entropy)


Epoch 1/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2948 - accuracy: 0.4229 - val_loss: 1.3760 - val_accuracy: 0.3327
Epoch 2/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2859 - accuracy: 0.4240 - val_loss: 1.3715 - val_accuracy: 0.3250
Epoch 3/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2819 - accuracy: 0.4252 - val_loss: 1.3663 - val_accuracy: 0.3351
Epoch 4/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2802 - accuracy: 0.4259 - val_loss: 1.3762 - val_accuracy: 0.3321
Epoch 5/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2793 - accuracy: 0.4261 - val_loss: 1.3879 - val_accuracy: 0.3343
Epoch 6/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2788 - accuracy: 0.4261 - val_loss: 1.3704 - val_accuracy: 0.3420
Epoch 7/10
6667/6667 [==============================] - 8s 1ms/step - loss: 1.2785 - accuracy: 0.4262 - val_loss: 1.3662 - val_accuracy:

In [200]:
# print(data)
# print(softmax_probs)
# print(y_onehot)

In [201]:
softmax_probs

array([[0.39794594, 0.3580835 , 0.05065088, 0.19331975],
       [0.14135127, 0.419494  , 0.14658503, 0.29256967],
       [0.14135127, 0.419494  , 0.14658503, 0.29256967],
       ...,
       [0.14135127, 0.419494  , 0.14658503, 0.29256967],
       [0.14135127, 0.419494  , 0.14658503, 0.29256967],
       [0.14135127, 0.419494  , 0.14658503, 0.29256967]], dtype=float32)

In [202]:
row_sums = np.sum(softmax_probs, axis=1)

In [203]:
row_sums

array([1.0000001, 1.       , 1.       , ..., 1.       , 1.       ,
       1.       ], dtype=float32)

In [204]:
unique_values = np.unique(row_sums)

# Printing values rounded till 6 decimal places
print([round(i, 6) for i in unique_values])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [205]:
y_onehot

<tf.Tensor: shape=(266675, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

### Computing for entire data

In [206]:
x = data[geometric_features].values

In [207]:
X = data[geometric_features].values
y = data['class'].values  # Get class labels

# Filter out classes not present in the dataset
num_classes = len(data['class'].unique())  # Update to the number of present classes (1, 2, 5, 8)
classes_present = [int(i) for i in data['class'].unique()]  # Classes present in the dataset
class_mapping = {cls: i for i, cls in enumerate(classes_present)}
y_mapped = np.array([class_mapping[cls] for cls in y])

# Convert class labels to one-hot encoding
y_onehot = tf.one_hot(y_mapped, depth=num_classes)


# Define the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(len(geometric_features),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(4, activation='softmax')  # Output layer with softmax activation for the present classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y_onehot, epochs=10, batch_size=32)

softmax_probs = model.predict(x)
entropy = -np.sum(softmax_probs * np.log(softmax_probs), axis=-1)

print("Per-point softmax entropy:", entropy)


Epoch 1/10
8334/8334 [==============================] - 8s 966us/step - loss: 1.3077 - accuracy: 0.4051
Epoch 2/10
8334/8334 [==============================] - 8s 949us/step - loss: 1.2985 - accuracy: 0.4091
Epoch 3/10
8334/8334 [==============================] - 8s 977us/step - loss: 1.2965 - accuracy: 0.4103
Epoch 4/10
8334/8334 [==============================] - 8s 967us/step - loss: 1.2957 - accuracy: 0.4103
Epoch 5/10
8334/8334 [==============================] - 8s 935us/step - loss: 1.2953 - accuracy: 0.4103
Epoch 6/10
8334/8334 [==============================] - 8s 939us/step - loss: 1.2948 - accuracy: 0.4104
Epoch 7/10
8334/8334 [==============================] - 8s 974us/step - loss: 1.2943 - accuracy: 0.4104
Epoch 8/10
8334/8334 [==============================] - 8s 953us/step - loss: 1.2941 - accuracy: 0.4103
Epoch 9/10
8334/8334 [==============================] - 8s 951us/step - loss: 1.2937 - accuracy: 0.4104
Epoch 10/10
8334/8334 [==============================] - 7s 823u

In [208]:
feature_densities = []
labels = [int(i) for i in data['class'].unique()]

for i in range (x.shape[0]):
    rel_probs = []
    deno = 0
    for label in labels:
        x1 = multivariate_gaussian_pdf(x[i], mu[label], sigma[label])
        deno += x1
        rel_probs.append(x1)
    probs = [x1/deno for x1 in rel_probs]
    feature_density = 0
    for j in range (len(labels)):
        feature_density += phi[labels[j]]*probs[j]
    feature_densities.append([x[i], feature_density])

C:\Users\Vidhish17\AppData\Local\Temp\ipykernel_4660\83269348.py:11: RuntimeWarning: invalid value encountered in scalar divide
  probs = [x1/deno for x1 in rel_probs]


In [209]:
labels

[2, 8, 1, 5]

In [210]:
data['feature_density'] = [x[1] for x in feature_densities]
data['softmax_probs'] = [x for x in softmax_probs]
data['entropy'] = entropy

In [211]:
data

,x,y,z,class,eigenvalues_sum,omnivariance,eigentropy,anisotropy,planarity,linearity,sphericity,verticality,feature_density,softmax_probs,entropy
0,0.001387,1.10728,0.017115,2.0,0.10524,0.01319,0.31369,0.98523,0.79733,0.18790,0.01477,0.00002,0.192130,"[0.39686155, 0.28776363, 0.08122798, 0.23414679]",1.269068
1,0.001387,1.10974,0.017115,2.0,0.10538,0.01320,0.31396,0.98526,0.79686,0.18839,0.01474,0.00002,0.192130,"[0.39831564, 0.28716394, 0.08070589, 0.23381454]",1.267866
2,0.001541,1.11221,0.017115,2.0,0.10552,0.01321,0.31424,0.98528,0.79624,0.18904,0.01472,0.00002,0.192130,"[0.40018854, 0.2864071, 0.080026716, 0.23337767]",1.266293
3,0.001541,1.11221,0.017115,2.0,0.10552,0.01321,0.31424,0.98528,0.79624,0.18904,0.01472,0.00002,0.192130,"[0.40018854, 0.2864071, 0.080026716, 0.23337767]",1.266293
4,0.003545,1.11714,0.017115,2.0,0.10593,0.01324,0.31502,0.98535,0.79561,0.18974,0.01465,0.00002,0.192130,"[0.40247092, 0.28537217, 0.07926737, 0.2328895]",1.264449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266670,1.066120,1.76079,0.021406,5.0,0.10721,0.01333,0.31740,0.98564,0.78459,0.20106,0.01436,0.00014,0.192129,"[0.40094736, 0.3067748, 0.07802903, 0.21424882]",1.258036
266671,1.066280,1.76079,0.017115,8.0,0.10719,0.01333,0.31737,0.98565,0.78448,0.20117,0.01435,0.00014,0.192129,"[0.40090486, 0.3070119, 0.07801785, 0.21406539]",1.257966
266672,1.066430,1.76326,0.017115,8.0,0.10699,0.01331,0.31700,0.98561,0.78687,0.19874,0.01439,0.00014,0.192131,"[0.4013549, 0.30219346, 0.078306094, 0.2181455]",1.259634
266673,1.066430,1.76572,0.017115,8.0,0.10677,0.01329,0.31659,0.98556,0.78986,0.19569,0.01444,0.00015,0.192131,"[0.4018129, 0.2961559, 0.07865931, 0.22337182]",1.261561


In [212]:
data['feature_density'].describe()

count    266548.000000
mean          0.240034
std           0.020484
min           0.192109
25%           0.236779
50%           0.251728
75%           0.251728
max           0.259380
Name: feature_density, dtype: float64

In [213]:
data['entropy'].describe()

count    266675.000000
mean          1.289849
std           0.025749
min           1.151420
25%           1.298445
50%           1.299515
75%           1.299515
max           1.362504
Name: entropy, dtype: float64

In [214]:
data.to_csv(f"CloudCompare/r{scale}-computed.csv", index=False)